# Overview

## Database access

Clean credit card data so it can be pushed to the database  
Data from: https://www.consumerfinance.gov/data-research/credit-card-data/

In [18]:
from sshtunnel import SSHTunnelForwarder

# SSH config shortcut for AWS
SSH_CONFIG_SHORTCUT = 'myaws'

server = SSHTunnelForwarder(
    SSH_CONFIG_SHORTCUT,
    ssh_config_file='~/.ssh/config',
    remote_bind_address=('localhost', 5432)
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 58012


In [19]:
from sqlalchemy import create_engine
###SQL Server access

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = '**********'  # Like I'm gonna give you my access credentials...
POSTGRES_PASSWORD = '**********'
POSTGRES_DBNAME = 'cfpb'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [3]:
import pandas as pd

df = pd.read_csv('./data/Survey_of_Credit_Card_Plans.csv')

In [25]:
df.columns = df.columns.str.replace(' ', '_')

In [27]:
#Parse numerical columns so they can be read as numbers in SQL
df['APR'] = df['APR'].apply(lambda x: float(str(x).replace('%', '')))
df['Annual_Fee'] = df['Annual_Fee'].apply(lambda x: float(str(x).replace('$', '')))
df['Late_Payment_Fee'] = df['Late_Payment_Fee'].apply(lambda x: float(str(x).replace('$', '')))

In [28]:
df.to_sql('cc_plans', cnx, if_exists='replace', index=False)

In [30]:
server.close()